In [ ]:
# @title Install necessary packages.
!pip install --upgrade --no-cache-dir dopamine-rl
!pip install cmake
!pip install atari_py
!pip install gin-config

Requirement already up-to-date: dopamine-rl in /Users/margaridasousa/dopamine/venv/lib/python3.6/site-packages
Requirement already up-to-date: gin-config>=0.1.1 in /Users/margaridasousa/dopamine/venv/lib/python3.6/site-packages (from dopamine-rl)
Requirement already up-to-date: absl-py>=0.2.2 in /Users/margaridasousa/dopamine/venv/lib/python3.6/site-packages (from dopamine-rl)
Requirement already up-to-date: opencv-python>=3.4.1.15 in /Users/margaridasousa/dopamine/venv/lib/python3.6/site-packages (from dopamine-rl)
Requirement already up-to-date: gym>=0.10.5 in /Users/margaridasousa/dopamine/venv/lib/python3.6/site-packages (from dopamine-rl)
Requirement already up-to-date: six>=1.10.0 in /Users/margaridasousa/dopamine/venv/lib/python3.6/site-packages (from gin-config>=0.1.1->dopamine-rl)
Requirement already up-to-date: enum34>=1.1.6 in /Users/margaridasousa/dopamine/venv/lib/python3.6/site-packages (from gin-config>=0.1.1->dopamine-rl)
Requirement already up-to-date: numpy>=1.11.3 in

In [37]:
# @title Necessary imports and globals.

import numpy as np
import os
from dopamine.agents.implicit_quantile import implicit_quantile_agent
from dopamine.discrete_domains import run_experiment
from dopamine.colab import utils as colab_utils
from absl import flags
import gin.tf

BASE_PATH = '/tmp/colab_dope_run'  # @param
GAME = 'Asterix'  # @param

In [38]:
# @title Load baseline data
!gsutil -q -m cp -R gs://download-dopamine-rl/preprocessed-benchmarks/* /content/
experimental_data = colab_utils.load_baselines('/content')

/bin/sh: gsutil: command not found


In [39]:
# @title Load the configuration for DQN.

IQN_PATH = os.path.join(BASE_PATH, 'implicit_quantile')
# Modified from dopamine/agents/dqn/config/dqn_cartpole.gin
iqn_config = """

import dopamine.agents.implicit_quantile.implicit_quantile_agent
import dopamine.agents.rainbow.rainbow_agent
import dopamine.discrete_domains.atari_lib
import dopamine.discrete_domains.run_experiment
import dopamine.replay_memory.prioritized_replay_buffer
import gin.tf.external_configurables

ImplicitQuantileAgent.kappa = 1.0
ImplicitQuantileAgent.num_tau_samples = 64
ImplicitQuantileAgent.num_tau_prime_samples = 64
ImplicitQuantileAgent.num_quantile_samples = 32
RainbowAgent.gamma = 0.99
RainbowAgent.update_horizon = 1
RainbowAgent.min_replay_history = 500 # agent steps
RainbowAgent.update_period = 4
RainbowAgent.target_update_period = 100 # agent steps
RainbowAgent.epsilon_train = 0.01
RainbowAgent.epsilon_eval = 0.001
RainbowAgent.epsilon_decay_period = 250000  # agent steps
# IQN currently does not support prioritized replay.
RainbowAgent.replay_scheme = 'uniform'
RainbowAgent.tf_device = '/gpu:0'  # '/cpu:*' use for non-GPU version
RainbowAgent.optimizer = @tf.train.AdamOptimizer()

tf.train.AdamOptimizer.learning_rate = 0.00005
tf.train.AdamOptimizer.epsilon = 0.0003125

atari_lib.create_atari_environment.game_name = 'Pong'
# Sticky actions with probability 0.25, as suggested by (Machado et al., 2017).
atari_lib.create_atari_environment.sticky_actions = True
create_agent.agent_name = 'implicit_quantile'
Runner.num_iterations = 50
Runner.training_steps = 1000
Runner.evaluation_steps = 1000
Runner.max_steps_per_episode = 200  # Default max episode length.

WrappedPrioritizedReplayBuffer.replay_capacity = 50000
WrappedPrioritizedReplayBuffer.batch_size = 128
"""
gin.parse_config(iqn_config, skip_unknown=False)

In [40]:
# @title Train IQN 
iqn_runner = run_experiment.create_runner(IQN_PATH, schedule='continuous_train')
print('Will train DQN agent, please be patient, may be a while...')
iqn_runner.run_experiment()
print('Done training!')

INFO:tensorflow:Creating TrainRunner ...
INFO:tensorflow:Creating ImplicitQuantileAgent agent with the following parameters:
INFO:tensorflow:	 gamma: 0.990000
INFO:tensorflow:	 update_horizon: 1.000000
INFO:tensorflow:	 min_replay_history: 500
INFO:tensorflow:	 update_period: 4
INFO:tensorflow:	 target_update_period: 100
INFO:tensorflow:	 epsilon_train: 0.010000
INFO:tensorflow:	 epsilon_eval: 0.001000
INFO:tensorflow:	 epsilon_decay_period: 250000
INFO:tensorflow:	 tf_device: /gpu:0
INFO:tensorflow:	 use_staging: True
INFO:tensorflow:	 optimizer: <tensorflow.python.training.adam.AdamOptimizer object at 0x2f2df7ef0>
INFO:tensorflow:Creating a OutOfGraphPrioritizedReplayBuffer replay memory with the following parameters:
INFO:tensorflow:	 observation_shape: (84, 84)
INFO:tensorflow:	 observation_dtype: <class 'numpy.uint8'>
INFO:tensorflow:	 stack_size: 4
INFO:tensorflow:	 replay_capacity: 50000
INFO:tensorflow:	 batch_size: 128
INFO:tensorflow:	 update_horizon: 1
INFO:tensorflow:	 gamm

TypeError: Input 'y' of 'Add' Op has type float32 that does not match type int32 of argument 'x'.

originally defined at:
  File "/Users/margaridasousa/dopamine/dopamine/agents/implicit_quantile/implicit_quantile_agent.py", line 142, in _build_networks
    self.online_convnet = tf.make_template('Online', self._network_template)
  File "/Users/margaridasousa/dopamine/venv/lib/python3.6/site-packages/tensorflow/python/ops/template.py", line 154, in make_template
    **kwargs)

  In call to configurable 'DQNAgent' (<function DQNAgent.__init__ at 0x1348db6a8>)
  In call to configurable 'RainbowAgent' (<function RainbowAgent.__init__ at 0x1348ca400>)
  In call to configurable 'ImplicitQuantileAgent' (<function ImplicitQuantileAgent.__init__ at 0x1348d51e0>)
  In call to configurable 'create_agent' (<function create_agent at 0x1348d5d08>)
  In call to configurable 'Runner' (<function Runner.__init__ at 0x1348e6378>)
  In call to configurable 'TrainRunner' (<function TrainRunner.__init__ at 0